In [1]:
#Import necessary packages
import pandas as pd
import seaborn as sns
%pylab inline
import matplotlib.pyplot as plt
import requests, re
import pandas as pd
import seaborn as sns
import nltk
import string, itertools
from collections import Counter, defaultdict
from nltk.text import Text
from nltk.probability import FreqDist
from nltk.tokenize import word_tokenize, sent_tokenize, regexp_tokenize
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer, WordNetLemmatizer
from gensim.corpora.dictionary import Dictionary
from gensim.models.tfidfmodel import TfidfModel
from sklearn.cluster import KMeans
from wordcloud import WordCloud

Populating the interactive namespace from numpy and matplotlib


In [3]:
#Load the detail dataset
detail = pd.read_excel('Q3_competition_detail_dataset.xlsx')
detail.head()

,id,name,is_claimed,is_closed,phone,review_count,categories01,categories02,categories03,rating,price,transactions,zip_code,city,address,restaurant_url,image_url,latitude,longitude,photos,cross_streets
0,nzgC5hhlnSq2DYbJbtH5MQ,Foxy's Landing & Restaurant,True,False,1.661949e+10,52,breakfast_brunch,tradamerican,NaN,4.0,$$,[],93536.0,Lancaster,"['4555 W Avenue G', 'Lancaster, CA 93536']",https://www.yelp.com/biz/foxys-landing-and-res...,https://s3-media1.fl.yelpcdn.com/bphoto/LooWtz...,34.738829,-118.216215,['https://s3-media1.fl.yelpcdn.com/bphoto/LooW...,NaN
1,i-2aG9_PQBEy7LrsRv0Ivg,Mosman's Steakhouse,True,False,1.661949e+10,63,bars,steak,NaN,3.5,$$,[],93536.0,Lancaster,"['46645 W 60th W', 'Lancaster, CA 93536']",https://www.yelp.com/biz/mosmans-steakhouse-la...,https://s3-media3.fl.yelpcdn.com/bphoto/JJ3mkC...,34.730580,-118.238360,['https://s3-media3.fl.yelpcdn.com/bphoto/JJ3m...,NaN
2,DJoeogRsOW5s9MzgveHQ2A,El Tamarindo,True,False,1.661723e+10,11,salvadoran,NaN,NaN,3.5,$,[],93534.0,Lancaster,"['551 W Ave I', 'Ste E', 'Lancaster, CA 93534']",https://www.yelp.com/biz/el-tamarindo-lancaste...,https://s3-media1.fl.yelpcdn.com/bphoto/UMSkfH...,34.707469,-118.146286,['https://s3-media1.fl.yelpcdn.com/bphoto/UMSk...,NaN
3,hwWfv3sSxV3a47UAdSVT5w,Subway,True,False,1.661730e+10,13,sandwiches,NaN,NaN,2.5,$,"['delivery', 'pickup']",93535.0,Lancaster,"['1821 W Ave I', 'Unit 103', 'Lancaster, CA 93...",https://www.yelp.com/biz/subway-lancaster-106?...,https://s3-media3.fl.yelpcdn.com/bphoto/4Yo7Ea...,34.705218,-118.164180,['https://s3-media3.fl.yelpcdn.com/bphoto/4Yo7...,NaN
4,TxU0fwF2N2nVhCpzokc1Pg,Little Caesars,True,False,1.661946e+10,54,pizza,NaN,NaN,1.5,$,[],93534.0,Lancaster,"['1841 W Ave I', 'Bldg 2, Ste D', 'Lancaster, ...",https://www.yelp.com/biz/little-caesars-lancas...,https://s3-media2.fl.yelpcdn.com/bphoto/-Fz1W5...,34.705100,-118.164870,['https://s3-media2.fl.yelpcdn.com/bphoto/-Fz1...,NaN


In [4]:
#Combine all 3 categories into 1 column
detail['categories'] = detail[['categories01', 'categories02', 'categories03']].apply(
    lambda x: ','.join(x.dropna().astype(str)),
    axis=1
)
detail.head()

,id,name,is_claimed,is_closed,phone,review_count,categories01,categories02,categories03,rating,price,transactions,zip_code,city,address,restaurant_url,image_url,latitude,longitude,photos,cross_streets,categories
0,nzgC5hhlnSq2DYbJbtH5MQ,Foxy's Landing & Restaurant,True,False,1.661949e+10,52,breakfast_brunch,tradamerican,NaN,4.0,$$,[],93536.0,Lancaster,"['4555 W Avenue G', 'Lancaster, CA 93536']",https://www.yelp.com/biz/foxys-landing-and-res...,https://s3-media1.fl.yelpcdn.com/bphoto/LooWtz...,34.738829,-118.216215,['https://s3-media1.fl.yelpcdn.com/bphoto/LooW...,NaN,"breakfast_brunch,tradamerican"
1,i-2aG9_PQBEy7LrsRv0Ivg,Mosman's Steakhouse,True,False,1.661949e+10,63,bars,steak,NaN,3.5,$$,[],93536.0,Lancaster,"['46645 W 60th W', 'Lancaster, CA 93536']",https://www.yelp.com/biz/mosmans-steakhouse-la...,https://s3-media3.fl.yelpcdn.com/bphoto/JJ3mkC...,34.730580,-118.238360,['https://s3-media3.fl.yelpcdn.com/bphoto/JJ3m...,NaN,"bars,steak"
2,DJoeogRsOW5s9MzgveHQ2A,El Tamarindo,True,False,1.661723e+10,11,salvadoran,NaN,NaN,3.5,$,[],93534.0,Lancaster,"['551 W Ave I', 'Ste E', 'Lancaster, CA 93534']",https://www.yelp.com/biz/el-tamarindo-lancaste...,https://s3-media1.fl.yelpcdn.com/bphoto/UMSkfH...,34.707469,-118.146286,['https://s3-media1.fl.yelpcdn.com/bphoto/UMSk...,NaN,salvadoran
3,hwWfv3sSxV3a47UAdSVT5w,Subway,True,False,1.661730e+10,13,sandwiches,NaN,NaN,2.5,$,"['delivery', 'pickup']",93535.0,Lancaster,"['1821 W Ave I', 'Unit 103', 'Lancaster, CA 93...",https://www.yelp.com/biz/subway-lancaster-106?...,https://s3-media3.fl.yelpcdn.com/bphoto/4Yo7Ea...,34.705218,-118.164180,['https://s3-media3.fl.yelpcdn.com/bphoto/4Yo7...,NaN,sandwiches
4,TxU0fwF2N2nVhCpzokc1Pg,Little Caesars,True,False,1.661946e+10,54,pizza,NaN,NaN,1.5,$,[],93534.0,Lancaster,"['1841 W Ave I', 'Bldg 2, Ste D', 'Lancaster, ...",https://www.yelp.com/biz/little-caesars-lancas...,https://s3-media2.fl.yelpcdn.com/bphoto/-Fz1W5...,34.705100,-118.164870,['https://s3-media2.fl.yelpcdn.com/bphoto/-Fz1...,NaN,pizza


In [5]:
#Load review table
review = pd.read_excel('Q3_competition_review_dataset.xlsx')
review.head()

,id,review_id,review_text,review_rating,review_time_created
0,cal0Wpupxj9c_AV7WzDXsw,AyueC5Vq_5lUKJFqSzXWWw,Slightly turned off by the hostess. She wasn't...,3.0,2021-07-13 15:01:59
1,cal0Wpupxj9c_AV7WzDXsw,yaH4AmHUz9b3Ywv4VtvU5g,Wish I would have known about no brunch at the...,3.0,2021-07-06 13:50:42
2,cal0Wpupxj9c_AV7WzDXsw,YiuFLFWsrP92_QWa-d2W2Q,I had an amazing experience at Granville.\n\nw...,5.0,2021-08-09 21:06:24
3,jVYU9iXvBMiC2A4H12Azfg,VyKvwjOuJxKWiLlyzsqQ_A,Photo dump from dinner on Aug 8th. Literally c...,5.0,2021-08-09 13:01:09
4,jVYU9iXvBMiC2A4H12Azfg,D0-MjyINO2u9IRmf1opaUQ,I've had this place bookmarked on my Yelp for ...,2.0,2021-07-17 16:28:47


In [6]:
#Check missing values
review.isnull().sum()

id                        6
review_id              1485
review_text            1485
review_rating          1485
review_time_created    1485
dtype: int64

### Merge two datasets and get new dataframe restaurants_reviews

In [7]:
#Merge detail and review datasets
restaurants_reviews = pd.merge(detail, review, on = 'id')

#Update column names
restaurants_reviews.rename(columns={'stars_x':'avg_star','stars_y':'review_star'}, inplace=True)


In [8]:
#Label reviews as positive or negative
restaurants_reviews['labels'] = ''
restaurants_reviews.loc[restaurants_reviews.review_rating >=4, 'labels'] = 'positive'
restaurants_reviews.loc[restaurants_reviews.review_rating ==3, 'labels'] = 'neural'
restaurants_reviews.loc[restaurants_reviews.review_rating <3, 'labels'] = 'negative'

#Drop neutral reviews for easy analysis
restaurants_reviews.drop(restaurants_reviews[restaurants_reviews['labels'] =='neural'].index, axis=0, inplace=True)
restaurants_reviews.reset_index(drop=True, inplace=True)

restaurants_reviews.head()

,id,name,is_claimed,is_closed,phone,review_count,categories01,categories02,categories03,rating,price,transactions,zip_code,city,address,restaurant_url,image_url,latitude,longitude,photos,cross_streets,categories,review_id,review_text,review_rating,review_time_created,labels
0,nzgC5hhlnSq2DYbJbtH5MQ,Foxy's Landing & Restaurant,True,False,1.661949e+10,52,breakfast_brunch,tradamerican,NaN,4.0,$$,[],93536.0,Lancaster,"['4555 W Avenue G', 'Lancaster, CA 93536']",https://www.yelp.com/biz/foxys-landing-and-res...,https://s3-media1.fl.yelpcdn.com/bphoto/LooWtz...,34.738829,-118.216215,['https://s3-media1.fl.yelpcdn.com/bphoto/LooW...,NaN,"breakfast_brunch,tradamerican",ncf3pKCumFbKOraN41iKWg,First time here in a LONNNGGGGGG time. Great f...,5.0,2021-06-01 16:31:19,positive
1,nzgC5hhlnSq2DYbJbtH5MQ,Foxy's Landing & Restaurant,True,False,1.661949e+10,52,breakfast_brunch,tradamerican,NaN,4.0,$$,[],93536.0,Lancaster,"['4555 W Avenue G', 'Lancaster, CA 93536']",https://www.yelp.com/biz/foxys-landing-and-res...,https://s3-media1.fl.yelpcdn.com/bphoto/LooWtz...,34.738829,-118.216215,['https://s3-media1.fl.yelpcdn.com/bphoto/LooW...,NaN,"breakfast_brunch,tradamerican",8huhMy9nMYEEoG8ANCnyLA,Came here on a whim because I was hungry and c...,5.0,2021-05-22 14:09:50,positive
2,nzgC5hhlnSq2DYbJbtH5MQ,Foxy's Landing & Restaurant,True,False,1.661949e+10,52,breakfast_brunch,tradamerican,NaN,4.0,$$,[],93536.0,Lancaster,"['4555 W Avenue G', 'Lancaster, CA 93536']",https://www.yelp.com/biz/foxys-landing-and-res...,https://s3-media1.fl.yelpcdn.com/bphoto/LooWtz...,34.738829,-118.216215,['https://s3-media1.fl.yelpcdn.com/bphoto/LooW...,NaN,"breakfast_brunch,tradamerican",sJhFp5G05kIdUvgyNCiLOQ,This place is so good!!!! Missed it during Cov...,5.0,2021-03-28 10:31:44,positive
3,i-2aG9_PQBEy7LrsRv0Ivg,Mosman's Steakhouse,True,False,1.661949e+10,63,bars,steak,NaN,3.5,$$,[],93536.0,Lancaster,"['46645 W 60th W', 'Lancaster, CA 93536']",https://www.yelp.com/biz/mosmans-steakhouse-la...,https://s3-media3.fl.yelpcdn.com/bphoto/JJ3mkC...,34.730580,-118.238360,['https://s3-media3.fl.yelpcdn.com/bphoto/JJ3m...,NaN,"bars,steak",YmW1ZcryZ0gizU3OPycvmw,Our first visit and we liked it. We enjoyed th...,4.0,2021-05-04 18:17:05,positive
4,i-2aG9_PQBEy7LrsRv0Ivg,Mosman's Steakhouse,True,False,1.661949e+10,63,bars,steak,NaN,3.5,$$,[],93536.0,Lancaster,"['46645 W 60th W', 'Lancaster, CA 93536']",https://www.yelp.com/biz/mosmans-steakhouse-la...,https://s3-media3.fl.yelpcdn.com/bphoto/JJ3mkC...,34.730580,-118.238360,['https://s3-media3.fl.yelpcdn.com/bphoto/JJ3m...,NaN,"bars,steak",GKcQeTQCQGkkvehogyA8PQ,We had heard good things about this steakhouse...,2.0,2021-06-20 02:00:48,negative


In [9]:
#Import packages for filtering positive and negative words
import csv
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.svm import LinearSVC

In [10]:
#Convert text to lower case
restaurants_reviews.review_text = restaurants_reviews.review_text.str.lower()

#Remove unnecessary punctuation
restaurants_reviews['removed_punct_text']= restaurants_reviews.review_text.str.replace('\n',''). \
                                          str.replace('[!"#$%&\()*+,-./:;<=>?@[\\]^_`{|}~]','')


In [11]:
#Import positive file which contains common meaningless positive words such as good
file_positive = open('positive.txt')
reader =csv.reader(file_positive)
positive_words = [word[0] for word in reader]

#Import negative file which contains common meaningless positive words such as bad
file_negative = open('negative.txt')
reader =csv.reader(file_negative)
negative_words = [word[0] for word in reader]

In [12]:
#Only keep positive and negative words
def filter_words(review):
    words = [word for word in review.split() if word in positive_words + negative_words]
    words = ' '.join(words)
    return words

In [13]:
#Drop rows with blank review
restaurants_reviews = restaurants_reviews[restaurants_reviews['review_text'].notna()]

In [15]:
#Filter words
restaurants_reviews['review_text'] = restaurants_reviews['review_text'].apply(filter_words)

In [16]:
#Split review_text into separate rows
restaurants_reviews2 = restaurants_reviews[['name', 'labels', 'review_text', 'categories']]
restaurants_reviews2.head()
restaurants_reviews2['review_text'] = restaurants_reviews2['review_text'].str.split(expand=True).stack().reset_index(name='Word').Word

,name,labels,review_text,categories
0,Foxy's Landing & Restaurant,positive,great good love enjoying,"breakfast_brunch,tradamerican"
1,Foxy's Landing & Restaurant,positive,pretty,"breakfast_brunch,tradamerican"
2,Foxy's Landing & Restaurant,positive,missed,"breakfast_brunch,tradamerican"
3,Mosman's Steakhouse,positive,liked enjoyed picky,"bars,steak"
4,Mosman's Steakhouse,negative,good excited,"bars,steak"


In [ ]:
#Export the file
restaurants_reviews2.to_csv(r'Restaurants_reviews2.csv', index = False)